# Boost GNN performance
[Project: Advanced Topics in Machine Learning and Optimization]

Strategies to augmentation of graph data to boost the performances of Graph Neural Networks.


## Libraries
*   Pytorch Geometric
*   OGB
*   Pytorch Lightning
*   graph-tool

In [ ]:
%%capture
!echo "deb http://downloads.skewed.de/apt bionic main" >> /etc/apt/sources.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-matplotlib python3-cairo

In [ ]:
%%capture
!pip install -U torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-1.12.0+cu113.html 
!pip install -U git+https://github.com/pyg-team/pytorch_geometric.git
!pip install -U pytorch-lightning ogb

Import the main packages

In [ ]:
import os
import shutil
import time

import torch
import torch_geometric as pyg
import pytorch_lightning as pl
import graph_tool.all as gt
import numpy as np
import pandas as pd

/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/all.py:39: RuntimeWarning: Error importing draw module, proceeding nevertheless: No module named 'cairo._cairo'
  warnings.warn(msg, RuntimeWarning)


Enable graph-tool parallel computation

In [ ]:
gt.openmp_set_num_threads(os.cpu_count() * 2)
gt.openmp_set_schedule("auto")

## Utils
General utility functions

In [ ]:
def is_directed(data):
  """
  Compute if the OGB graph is directed by checking if the edges are bidirectional.
  Faster than the standard OGB implementation.
  """
  if data.num_edges % 2 == 0:
    # Exploit adjacency of bidirectional (undirected) edges in OGB graphs
    idx = torch.arange(data.num_edges) + 1
    idx[torch.arange(1, data.num_edges, 2)] -= 2
    directed = (data.edge_index[0, idx] != data.edge_index[1]).any().item()
  else:
    directed = True
  
  return directed

In [ ]:
def collate(l):
  "Concatenate list of dictionaries of tensors. Take keys from first dict"
  def _collate(l, k):
    tensors = [d[k] for d in l]
    return torch.stack(tensors) if tensors[0].dim() == 0 else torch.cat(tensors)

  return {k: _collate(l, k) for k in l[0].keys()}

## Transforms
Define the data transformation to boost the performance of GNNs

In [ ]:
class FloatFeatures(pyg.transforms.BaseTransform):
  """
  Change all features datatype
  """
  def __call__(self, data):
    data.x = data.x.float()
    if data.edge_weight is not None:
      data.edge_weight = data.edge_weight.float()
    if data.edge_attr is not None:
      data.edge_attr = data.edge_attr.float()
    return data

In [ ]:
class StandardizeFeatures(pyg.transforms.BaseTransform):
  """
  Feature-wise standardization
  """
  def __call__(self, data, eps=1e-12):
    data.x = (data.x - data.x.mean(0)) / (data.x.std(0) + eps)
    data.x[data.x.isnan()] = 0
    if data.edge_attr is not None:
      data.edge_attr = (data.edge_attr - data.edge_attr.mean(0)) / (data.edge_attr.std(0) + eps)
      data.edge_attr[data.edge_attr.isnan()] = 0
    return data

In [ ]:
class DummyFeatures(pyg.transforms.BaseTransform):
  """
  A data transforms class that adds extra dummy nodes features
    num_features (int): number of dummy features
    normal (bool): If True, the features are sampled from a normal distribution
      If False, the features are ones. Default: True
  """
  def __init__(self, num_features=1, normal=True):
    super().__init__()
    self.num_features = num_features
    self.normal = normal

  def __call__(self, data):
    shape = (data.num_nodes, self.num_features)
    dummy_features = torch.randn(shape) if self.normal else torch.ones(shape)
    if 'x' in data:
      data.x = torch.hstack([data.x, dummy_features])
    else:
      data.x = dummy_features
    return data

In [ ]:
class TopologyFeatures(pyg.transforms.BaseTransform):
  """
  A data transforms class that adds new topology features to the nodes.
  Keeps track of the number of additional features.
  """
  def __init__(self):
    self.num_feats = 11

  @staticmethod
  def _to_tensor(feature):
    feature[np.isnan(feature)] = 0.
    return torch.from_numpy(feature.astype('float32'))

  def __call__(self, data):
    # Convert to a graph-tool graph
    directed = is_directed(data)
    g = gt.Graph(directed=directed)
    edges = data.edge_index.T.numpy()
    if not directed:
      # Remove duplicate edges (keep dimension)
      edges = edges[np.arange(0, len(edges), 2)]
    g.add_vertex(n=data.num_nodes)
    g.add_edge_list(edges)
    ## ====TOPOLOGY FEATURES====
    # In degree
    in_deg = g.get_in_degrees(g.get_vertices())
    # Out degree
    out_deg = g.get_out_degrees(g.get_vertices())
    # Local clustering coefficient
    clust = gt.local_clustering(g).a
    # Average NN in degree
    in_degrees = [g.get_in_degrees(g.get_all_neighbors(n)) for n in g.iter_vertices()]
    avg_nn_in_deg = np.array([deg.mean() if len(deg) > 0 else 0. for deg in in_degrees])
    # Average NN out degree
    out_degrees = [g.get_out_degrees(g.get_all_neighbors(n)) for n in g.iter_vertices()]
    avg_nn_out_deg = np.array([deg.mean() if len(deg) > 0 else 0. for deg in out_degrees])
    # Pagerank centrality
    pagerank = gt.pagerank(g, max_iter=1000).a
    # Eigenvector centrality
    _, eigen = gt.eigenvector(g, max_iter=1000)
    eigen = eigen.a
    # HITS centrality (authority and hub centralities)
    _, auth, hub = gt.hits(g, max_iter=1000)
    auth = auth.a
    hub = hub.a
    # Katz centrality
    katz = gt.katz(g, max_iter=1000).a
    # K-core decomposition
    kcore = gt.kcore_decomposition(g).a
    ## =========================
    features = [in_deg, out_deg, clust, avg_nn_in_deg, avg_nn_out_deg,
                pagerank, eigen, auth, hub, katz, kcore]
    # Add topology features to the nodes
    features = self._to_tensor(np.stack(features)).T
    assert features.shape == (data.num_nodes, 11)
    if 'x' in data:
      data.x = torch.hstack([data.x, features])
    else:
      data.x = features
    return data

In [ ]:
def get_transforms(topology=False, scaling=True):
  add_feats = TopologyFeatures() if topology else DummyFeatures(TopologyFeatures().num_feats, normal=False)
  transforms = [add_feats, FloatFeatures()]
  if scaling:
    transforms.append(StandardizeFeatures())
  return pyg.transforms.Compose(transforms)

## Data
Define the data collection functions

In [ ]:
def process_info(info):
  if info['task type'] == 'binary classification':
    info['pred_channels'] = int(info['num tasks'])
    info['loss'] = 'binary'
  elif info['task type'] == 'multiclass classification':
    info['pred_channels'] = int(info['num classes'])
    info['loss'] = 'multiclass'
  elif info['task type'] == 'link prediction':
    info['pred_channels'] = 1
    info['loss'] = 'binary'

In [ ]:
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator as NodeEvaluator
from torch_geometric.data import LightningNodeData

def get_node_data(name, batch_size, num_workers, num_neighbors, topology, scaling):
  dataset = PygNodePropPredDataset(name, transform=get_transforms(topology, scaling))
  data = dataset[0]
  # Datamodule
  split_idx = dataset.get_idx_split()
  datamodule = LightningNodeData(
    data,
    split_idx['train'],
    split_idx['valid'],
    split_idx['test'],
    num_neighbors=num_neighbors,
    batch_size=batch_size,
    num_workers=num_workers,
    pin_memory=True)
  # Evaluator
  evaluator = NodeEvaluator(name)
  # Info
  info = dataset.meta_info
  info['task'] = 'node'
  info['num_features'] = data.num_features
  info['num_edge_features'] = data.num_edge_features
  process_info(info)

  return datamodule, evaluator, info

In [ ]:
def process_edge_split(edge_split, data):
  # Sample the negative training edges
  edge_split['train']['edge_neg'] = pyg.utils.negative_sampling(
      data.edge_index,
      data.num_nodes,
      len(edge_split['train']['edge'])).t()
  # Aggregate together negative and positive edges
  processed_split = {}
  for split in ['train', 'valid', 'test']:
    edge = edge_split[split]['edge']
    edge_label = torch.ones(len(edge))
    neg = edge_split[split]['edge_neg']
    neg_label = torch.zeros(len(neg))
    processed_split[split] = {
        'edge': torch.cat([edge, neg]).t(),
        'label': torch.cat([edge_label, neg_label]).t()
    }
  return processed_split

In [ ]:
from ogb.linkproppred import PygLinkPropPredDataset, Evaluator as LinkEvaluator
from torch_geometric.data import LightningLinkData

def get_link_data(name, batch_size, num_workers, num_neighbors, topology, scaling):
  dataset = PygLinkPropPredDataset(name, transform=get_transforms(topology, scaling))
  data = dataset[0]
  # Datamodule
  edge_split = process_edge_split(dataset.get_edge_split(), dataset.data)
  datamodule = LightningLinkData(
    data,
    input_train_edges=edge_split['train']['edge'],
    input_val_edges=edge_split['valid']['edge'],
    input_test_edges=edge_split['test']['edge'],
    input_train_labels=edge_split['train']['label'],
    input_val_labels=edge_split['valid']['label'],
    input_test_labels=edge_split['test']['label'],
    num_neighbors=num_neighbors,
    batch_size=batch_size,
    num_workers=num_workers,
    pin_memory=True)
  # Evaluator
  evaluator = LinkEvaluator(name)
  # Info
  info = dataset.meta_info
  info['task'] = 'link'
  info['num_features'] = data.num_features
  info['num_edge_features'] = data.num_edge_features
  process_info(info)

  return datamodule, evaluator, info

In [ ]:
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator as GraphEvaluator
from torch_geometric.data import LightningDataset

def get_graph_data(name, batch_size, num_workers, topology, scaling):
  dataset = PygGraphPropPredDataset(name, pre_transform=get_transforms(topology, scaling))
  split_idx = dataset.get_idx_split()
  # Datamodule
  datamodule = LightningDataset(
    dataset[split_idx["train"]],
    dataset[split_idx["valid"]],
    dataset[split_idx["test"]],
    batch_size=batch_size,
    num_workers=num_workers,
    pin_memory=True)
  # Evaluator
  evaluator = GraphEvaluator(name)
  # Info
  info = dataset.meta_info
  info['task'] = 'graph'
  info['num_features'] = dataset.num_features
  info['num_edge_features'] = dataset.num_edge_features
  process_info(info)
  return datamodule, evaluator, info

In [ ]:
def get_data(name, batch_size=1024, num_workers=None, num_neighbors=[15,10,5], topology=False, scaling=True):
  num_workers = os.cpu_count() if num_workers is None else num_workers
  if name.startswith('ogbn'):
    return get_node_data(name, batch_size, num_workers, num_neighbors, topology, scaling)
  elif name.startswith('ogbl'):
    return get_link_data(name, batch_size, num_workers, num_neighbors, topology, scaling)
  elif name.startswith('ogbg'):
    return get_graph_data(name, batch_size, num_workers, topology, scaling)
  else:
    raise ValueError(f'Unknown dataset type: {name}')

## Models
* **GCN**: spectral method
* **GraphSAGE**: spatial method
* **GAT**: attentional (spatial) method


In [ ]:
class LightningGNN(pl.LightningModule):
  def __init__(self, evaluator, info, lr_min, lr_max):
    super().__init__()
    self.evaluator = evaluator
    self.lr_min = lr_min
    self.lr_max = lr_max
    self.task = info['task']
    self.loss_type = info['loss']

    if self.loss_type == 'binary':
      self.loss = torch.nn.BCEWithLogitsLoss()
    elif self.loss_type == 'multiclass':
      self.loss = torch.nn.CrossEntropyLoss()
    else:
      raise ValueError('Unknown loss type: ', self.loss_type)
  
  def configure_optimizers(self):
    optimizer = torch.optim.RAdam(self.parameters(), lr=self.lr_max, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, self.trainer.max_epochs, eta_min=self.lr_min)
    # scheduler = {"scheduler": torch.optim.lr_scheduler.OneCycleLR(
    #                 optimizer, max_lr=self.lr_max,
    #                 total_steps=self.trainer.estimated_stepping_batches), 
    #              "interval": "step"}
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 5)
    return [optimizer], [scheduler]
  
  def _get_target(self, batch):
    # Task type
    if self.task == 'node':
      target = batch.y[:batch.batch_size]
    elif self.task == 'link':
      target = batch.edge_label
    elif self.task == 'graph':
      target = batch.y
    # Prediction type
    if self.loss_type == 'multiclass':
      target = target.flatten()
    elif self.loss_type == 'binary':
      target = target.float()
    return target
  
  def _compute_metrics(self, logits, target):
    if self.task == 'link':
      y_pred_pos = logits[target == 1]
      y_pred_neg = logits[target == 0]
      return self.evaluator.eval({'y_pred_pos': y_pred_pos, 'y_pred_neg': y_pred_neg})
    else:
      if self.loss_type == 'multiclass':
        target = target.unsqueeze(-1)
        logits = logits.argmax(dim=-1, keepdim=True)
      return self.evaluator.eval({"y_true": target, "y_pred": logits})
  
  def log_metrics(self, metrics, prefix=None, batch_size=None):
    if prefix is not None:
      metrics = {f"{prefix}/{k}": v for k, v in metrics.items()}
    self.log_dict(metrics, batch_size=batch_size)
  
  def _step(self, batch):
    logits = self(batch)
    target = self._get_target(batch)
    loss = self.loss(logits, target)
    return logits, target, loss

  def training_step(self, batch, batch_idx):
    logits, target, loss = self._step(batch)
    self.log(f"train/loss", loss, batch_size=target.size(0))
    return {'logits': logits, 'target': target, 'loss': loss}

  def validation_step(self, batch, batch_idx):
    logits, target, loss = self._step(batch)
    self.log(f"valid/loss", loss, batch_size=target.size(0))
    return {'logits': logits, 'target': target}
  
  def test_step(self, batch, batch_idx):
    logits, target, loss = self._step(batch)
    self.log(f"test/loss", loss, batch_size=target.size(0))
    return {'logits': logits, 'target': target}
  
  def training_epoch_end(self, outs):
    outs = collate(outs)
    metrics = self._compute_metrics(outs['logits'], outs['target'])
    self.log_metrics(metrics, 'train')
  
  def validation_epoch_end(self, outs):
    outs = collate(outs)
    metrics = self._compute_metrics(outs['logits'], outs['target'])
    self.log_metrics(metrics, 'valid')
  
  def test_epoch_end(self, outs):
    outs = collate(outs)
    metrics = self._compute_metrics(outs['logits'], outs['target'])
    self.log_metrics(metrics, 'test')

In [ ]:
# Standard link predictor (unused)

class DotProductLinkPredictor():
  def __call__(self, z, edges):
    return (z[edges[0]] * z[edges[1]]).sum(-1)
  
  def decode_all(self, z):
    return z @ z.t()

In [ ]:
class HadamardLinkPredictor(torch.nn.Module):
  def __init__(self, emb_channels):
    super().__init__()
    self.linear = torch.nn.Linear(emb_channels, 1)
  
  def forward(self, z, edges):
    z_edges = z[edges[0]] * z[edges[1]]
    return self.linear(z_edges).flatten()

In [ ]:
class SharedModel(torch.nn.Module):
  def __init__(self, model, in_channels, hidden_channels, num_layers, out_channels, edge_dim, dropout, pooling=False):
    super().__init__()
    self.model = model
    self.in_channels = in_channels
    self.hidden_channels = hidden_channels
    self.num_layers = num_layers
    self.out_channels = out_channels
    self.edge_dim = edge_dim
    self.dropout = dropout
    self.pooling = pooling

    channels = [self.in_channels, *[self.hidden_channels] * self.num_layers]

    self.layers = torch.nn.ModuleList([
        pyg.nn.Sequential('x, edge_index, edge_attr, edge_weight', [
            self._conv_layer(in_ch, out_ch),
            pyg.nn.BatchNorm(out_ch),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(self.dropout)])
        for in_ch, out_ch in zip(channels[:-1], channels[1:])])

    if self.pooling:
      self.pools = torch.nn.ModuleList([self._pool_layer(self.hidden_channels) for _ in range(self.num_layers)])
      aggrs = ['mean', 'max']
      self.readout = pyg.nn.aggr.MultiAggregation(aggrs)
      emb_channels = self.hidden_channels * len(aggrs)
    else:
      self.pools = [None]*self.num_layers
      self.jk = pyg.nn.JumpingKnowledge('cat')
      emb_channels = self.hidden_channels * self.num_layers
    
    self.linear = pyg.nn.Linear(emb_channels, self.out_channels)
  
  def _pool_layer(self, channels):
    return pyg.nn.SAGPooling(channels, min_score=0.04, GNN=pyg.nn.SAGEConv, project=True)
    # return pyg.nn.ASAPooling(channels)
    # return pyg.nn.PANPooling(channels)
    # return pyg.nn.DMoNPooling(channels, k=16, dropout=self.dropout)

  def _conv_layer(self, in_channels, out_channels):
    if self.model == 'GCN':
      return (pyg.nn.GCNConv(in_channels, out_channels, improved=True),
              'x, edge_index, edge_weight -> x')
    elif self.model == 'GraphSAGE':
      return (pyg.nn.SAGEConv(in_channels, out_channels, project=True),
              'x, edge_index -> x')
    elif self.model == 'GAT':
      heads = 2
      return (pyg.nn.GATv2Conv(in_channels, out_channels // heads, edge_dim=self.edge_dim, heads=heads),
              'x, edge_index, edge_attr -> x')

  def forward(self, data):
    # General data
    x, edge_index = data.x, data.edge_index
    # Optional data
    edge_weight, edge_attr = data.edge_weight, data.edge_attr
    # Graph batch data
    batch = data.batch
    # Layers (+ pooling)
    xs = []
    for layer, pool in zip(self.layers, self.pools):
      x = layer(x, edge_index, edge_attr, edge_weight)
      if self.pooling:
        x, edge_index, edge_attr, batch, _, _ = pool(x, edge_index, edge_attr, batch)
        xs.append(self.readout(x, batch))
      else:
        xs.append(x)
    # Final aggregation
    if self.pooling:
      x = torch.stack(xs).sum(0)
    else:
      x = self.jk(xs)
    # MLP
    x = self.linear(x)
    return x

In [ ]:
class GraphNN(LightningGNN):
  def __init__(self, model, evaluator, info, *args, **kwargs):
    super().__init__(evaluator, info, *args, **kwargs)

    if info.num_edge_features > 0:
      self.edge_dim = info.num_edge_features
    else:
      self.edge_dim = None
    self.in_channels = info.num_features
    self.h_channels = 256
    self.out_channels = self.h_channels if self.task == 'link' else info['pred_channels']
    self.pooling = self.task == 'graph'

    self.net = SharedModel(model, self.in_channels, self.h_channels, 3, 
                           self.out_channels, self.edge_dim, 0.5, self.pooling)
    
    if self.task == 'link':
      self.linkpredictor = HadamardLinkPredictor(self.out_channels)

  def forward(self, data):
    logits = self.net(data)

    if self.task == 'node':
      return logits[:data.batch_size]
    elif self.task == 'link':
      return self.linkpredictor(logits, data.edge_label_index)
    elif self.task == 'graph':
      return logits

## Testing
Supplementary code for testing individual datasets and models

In [ ]:
if False:
  model = 'GAT'
  dataset_name = 'ogbg-molhiv'
  topology = False
  LR = 0.01
  EPOCHS = 15

  datamodule, evaluator, info = get_data(dataset_name, topology=topology, batch_size=1024)
  net = GraphNN(model, evaluator, info, LR/20, LR)
  trainer = pl.Trainer(accelerator='gpu' if torch.cuda.is_available() else 'cpu', 
                       max_epochs=EPOCHS,
                       precision=16 if torch.cuda.is_available() else 32,
                      #  auto_lr_find=True,
                      #  callbacks=[pl.callbacks.StochasticWeightAveraging(0.001)],
                       benchmark=True)
  # trainer.tune(net, datamodule=datamodule)
  # print(net.lr)
  s = time.time()
  trainer.fit(net, datamodule=datamodule)
  print("Execution time: ", time.time() - s)
  print(trainer.logged_metrics)
  trainer.test(net, datamodule=datamodule)
  print(trainer.logged_metrics)
  del datamodule, evaluator, info, trainer, net

In [ ]:
# %reload_ext tensorboard
# %tensorboard --logdir=lightning_logs/

In [ ]:
# # Remove all the runs logs
# !rm -rf lightning_logs

## Evaluation

In [ ]:
def evaluate(datasets, models=None, topologies=None, batch_size=1024, epochs=30, iters=1, lr=0.01, scaling=True):
  lr_min = lr / 20
  models = ['GCN', 'GraphSAGE', 'GAT'] if models is None else models
  topologies = [False, True] if topologies is None else topologies
  data = []
  for name in datasets:
    processed_path = os.path.join('dataset', name.replace('-', '_'), 'processed')
    for topology in topologies:
      if os.path.exists(processed_path):
        shutil.rmtree(processed_path)
      datamodule, evaluator, info = get_data(name, topology=topology, batch_size=batch_size, scaling=scaling)
      metric_name = evaluator.eval_metric
      for model in models:
        tr_metrics = []
        val_metrics = []
        test_metrics = []
        for _ in range(iters):
          trainer = pl.Trainer(
            accelerator='gpu' if torch.cuda.is_available() else 'cpu',
            precision=16 if torch.cuda.is_available() else 32,
            max_epochs=epochs,
            check_val_every_n_epoch=epochs,
            benchmark=True)
          net = GraphNN(model, evaluator, info, lr_min, lr)
          trainer.fit(net, datamodule=datamodule)
          tr_metrics.append(trainer.logged_metrics[f'train/{metric_name}'].item())
          val_metrics.append(trainer.logged_metrics[f'valid/{metric_name}'].item())
          trainer.test(net, datamodule=datamodule)
          test_metrics.append(trainer.logged_metrics[f'test/{metric_name}'].item())
          del trainer, net
        # End of experiment iterations
        metrics = torch.tensor([tr_metrics, val_metrics, test_metrics]).T
        print('Scores: ', metrics.tolist())
        # Get the best experiment according to validation
        best_exp = metrics[metrics[:, 1].argmax()]
        data.append({
          'dataset': name,
          'topology': topology,
          'model': model,
          'train': best_exp[0].item(),
          'val': best_exp[1].item(),
          'test': best_exp[2].item()
        })
        print(data)
      # End of same-model experiments
      del datamodule, evaluator, info
  return data

In [ ]:
results = []

In [ ]:
results.extend(evaluate(['ogbn-arxiv', 'ogbn-proteins']))
results.extend(evaluate(['ogbl-ddi', 'ogbl-collab'], batch_size=2048))
results.extend(evaluate(['ogbg-molhiv'], scaling=False, iters=3))

## Results

In [ ]:
# Collection of results from the experiments
results = [
    # Node datasets
    {'dataset': 'ogbn-arxiv', 'topology': False, 'model': 'GCN', 'train': 0.6486293077468872, 'val': 0.630658745765686, 'test': 0.5714461803436279}, 
    {'dataset': 'ogbn-arxiv', 'topology': False, 'model': 'GraphSAGE', 'train': 0.668664276599884, 'val': 0.6312292218208313, 'test': 0.5687509179115295}, 
    {'dataset': 'ogbn-arxiv', 'topology': False, 'model': 'GAT', 'train': 0.5814979076385498, 'val': 0.6245176196098328, 'test': 0.5686274766921997}, 
    {'dataset': 'ogbn-arxiv', 'topology': True, 'model': 'GCN', 'train': 0.6501578092575073, 'val': 0.6440820097923279, 'test': 0.5918359160423279}, 
    {'dataset': 'ogbn-arxiv', 'topology': True, 'model': 'GraphSAGE', 'train': 0.6702807545661926, 'val': 0.6432766318321228, 'test': 0.5800464749336243}, 
    {'dataset': 'ogbn-arxiv', 'topology': True, 'model': 'GAT', 'train': 0.5867870450019836, 'val': 0.6331420540809631, 'test': 0.5874740481376648},
    {'dataset': 'ogbn-proteins', 'topology': False, 'model': 'GCN', 'train': 0.5445547347931017, 'val': 0.4787758004087343, 'test': 0.4489186343826641}, 
    {'dataset': 'ogbn-proteins', 'topology': False, 'model': 'GraphSAGE', 'train': 0.5779571577772371, 'val': 0.49951851428883826, 'test': 0.5004665217140057}, 
    {'dataset': 'ogbn-proteins', 'topology': False, 'model': 'GAT', 'train': 0.49877383659463614, 'val': 0.5, 'test': 0.5},
    {'dataset': 'ogbn-proteins', 'topology': True, 'model': 'GCN', 'train': 0.749247411700721, 'val': 0.6870221382984857, 'test': 0.6474622706396195},
    {'dataset': 'ogbn-proteins', 'topology': True, 'model': 'GraphSAGE', 'train': 0.7875600519764958, 'val': 0.7705465701359874, 'test': 0.7214632246769581},
    {'dataset': 'ogbn-proteins', 'topology': True, 'model': 'GAT', 'train': 0.7274739553656298, 'val': 0.7115133268265658, 'test': 0.6516166219545089},
    # Link datasets
    {'dataset': 'ogbl-ddi', 'topology': False, 'model': 'GCN', 'train': 1.9664559658849612e-05, 'val': 0.0, 'test': 3.7456269637914374e-05}, 
    {'dataset': 'ogbl-ddi', 'topology': False, 'model': 'GraphSAGE', 'train': 1.3109706742397975e-05, 'val': 0.0, 'test': 0.0},
    {'dataset': 'ogbl-ddi', 'topology': False, 'model': 'GAT', 'train': 2.5283005015808158e-05, 'val': 0.0, 'test': 0.0},
    {'dataset': 'ogbl-ddi', 'topology': True, 'model': 'GCN', 'train': 0.0011620818404480815, 'val': 0.0412093885242939, 'test': 0.027904920279979706},
    {'dataset': 'ogbl-ddi', 'topology': True, 'model': 'GraphSAGE', 'train': 0.0012463585007935762, 'val': 0.1552487462759018, 'test': 0.16927985846996307},
    {'dataset': 'ogbl-ddi', 'topology': True, 'model': 'GAT', 'train': 0.0011527177412062883, 'val': 0.16681523621082306, 'test': 0.12424993515014648},
    {'dataset': 'ogbl-collab', 'topology': False, 'model': 'GCN', 'train': 0.14299708604812622, 'val': 0.30375808477401733, 'test': 0.2320145070552826},
    {'dataset': 'ogbl-collab', 'topology': False, 'model': 'GraphSAGE', 'train': 0.09535627067089081, 'val': 0.36107781529426575, 'test': 0.29115673899650574},
    {'dataset': 'ogbl-collab', 'topology': False, 'model': 'GAT', 'train': 0.0935022383928299, 'val': 0.2761467397212982, 'test': 0.25871485471725464},
    {'dataset': 'ogbl-collab', 'topology': True, 'model': 'GraphSAGE', 'train': 0.135894775390625, 'val': 0.3589308261871338, 'test': 0.3108851909637451},
    {'dataset': 'ogbl-collab', 'topology': True, 'model': 'GCN', 'train': 0.13747484982013702, 'val': 0.3091338872909546, 'test': 0.26670119166374207},
    {'dataset': 'ogbl-collab', 'topology': True, 'model': 'GAT', 'train': 0.13744856417179108, 'val': 0.42239198088645935, 'test': 0.35873860120773315},
    # Graph dataset
    {'dataset': 'ogbg-molhiv', 'topology': False, 'model': 'GCN', 'train': 0.7381002902984619, 'val': 0.7269743084907532, 'test': 0.7571409344673157}, 
    {'dataset': 'ogbg-molhiv', 'topology': False, 'model': 'GraphSAGE', 'train': 0.7013407349586487, 'val': 0.6859399676322937, 'test': 0.7036501169204712}, 
    {'dataset': 'ogbg-molhiv', 'topology': False, 'model': 'GAT', 'train': 0.7536150217056274, 'val': 0.7226080298423767, 'test': 0.7402392625808716}, 
    {'dataset': 'ogbg-molhiv', 'topology': True, 'model': 'GCN', 'train': 0.7316924929618835, 'val': 0.7352568507194519, 'test': 0.7333166003227234}, 
    {'dataset': 'ogbg-molhiv', 'topology': True, 'model': 'GraphSAGE', 'train': 0.7217303514480591, 'val': 0.6968664526939392, 'test': 0.7334536910057068}, 
    {'dataset': 'ogbg-molhiv', 'topology': True, 'model': 'GAT', 'train': 0.7461316585540771, 'val': 0.7006555795669556, 'test': 0.7179793119430542}
]

In [ ]:
index = ['dataset', 'model', 'topology']
pd.DataFrame(results).set_index(index).sort_values(by=index).round(3)

train    val   test
dataset       model     topology                     
ogbg-molhiv   GAT       False     0.754  0.723  0.740
                        True      0.746  0.701  0.718
              GCN       False     0.738  0.727  0.757
                        True      0.732  0.735  0.733
              GraphSAGE False     0.701  0.686  0.704
                        True      0.722  0.697  0.733
ogbl-collab   GAT       False     0.094  0.276  0.259
                        True      0.137  0.422  0.359
              GCN       False     0.143  0.304  0.232
                        True      0.137  0.309  0.267
              GraphSAGE False     0.095  0.361  0.291
                        True      0.136  0.359  0.311
ogbl-ddi      GAT       False     0.000  0.000  0.000
                        True      0.001  0.167  0.124
              GCN       False     0.000  0.000  0.000
                        True      0.001  0.041  0.028
              GraphSAGE False     0.000  0.000  0.000
                        True      0.001  0.155  0.169
ogbn-arxiv    GAT       False     0.581  0.625  0.569
                        True      0.587  0.633  0.587
              GCN       False     0.649  0.631  0.571
                        True      0.650  0.644  0.592
              GraphSAGE False     0.669  0.631  0.569
                        True      0.670  0.643  0.580
ogbn-proteins GAT       False     0.499  0.500  0.500
                        True      0.727  0.712  0.652
              GCN       False     0.545  0.479  0.449
                        True      0.749  0.687  0.647
              GraphSAGE False     0.578  0.500  0.500
                        True      0.788  0.771  0.721